In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from PIL import Image
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, Sampler
import numpy as np
import cv2 as cv
import torchlayers as tl
import os

In [ ]:
trans = transforms.Compose([transforms.Resize((120, 120)), transforms.ToTensor()])
trans_gray = transforms.Compose([transforms.Resize((120, 120)), transforms.ToTensor(), transforms.Grayscale()])


# datas = torchvision.datasets.ImageFolder(root = './data/kcar/lights', transform = trans)
# train_size = int(len(datas) * 0.8)
# test_size = len(datas) - train_size
# train_sets, test_sets = torch.utils.data.random_split(datas, [train_size, test_size])

edge_datas = torchvision.datasets.ImageFolder(root = './data/kcar_edge/lights', transform = trans_gray)
edge_train_size = int(len(edge_datas) * 0.8)
edge_test_size = len(edge_datas) - edge_train_size
edge_train_sets, edge_test_sets = torch.utils.data.random_split(edge_datas, [edge_train_size, edge_test_size])

# edge_train_sets = torchvision.datasets.ImageFolder(root = './data/kcar_edge/lights', transform = trans)
# edge_test_sets = torchvision.datasets.ImageFolder(root = './data/kcar_edge/darks', transform = trans)
train_sets = torchvision.datasets.ImageFolder(root = './data/kcar/lights', transform = trans)
test_sets = torchvision.datasets.ImageFolder(root = './data/kcar/darks', transform = trans)
labels = train_sets.classes
print(len(edge_train_sets))
print(len(edge_test_sets))
print(len(train_sets))
print(len(test_sets))
print(len(labels))
# print(test_sets)
# print(edge_test_sets)

# print(len(labels))
# for i in range(4):
#     img, label = train_sets[i]
#     print(labels[label])
#     plt.subplot(241 + i)
#     plt.imshow(np.clip(img.permute(1,2,0),0,1))
#     plt.tight_layout()
#     plt.show()
    # save_image(img, str(i)+'_.jpg')


In [ ]:
# DataLoader 정의 - data 길이 달라지면 random_t_list 수정하기!
import random
random_train_list = list(range(1, len(edge_train_sets)))
random.shuffle(random_train_list)
random_test_list = list(range(1, len(edge_test_sets)))
random.shuffle(random_test_list)

train_loader = DataLoader(train_sets, batch_size = 32, num_workers = 2, sampler = random_train_list)
edge_train_loader = DataLoader(edge_train_sets, batch_size = 32, num_workers = 2, sampler = random_train_list)
print(len(train_loader))
test_loader = DataLoader(test_sets, batch_size = 32, sampler = random_test_list)
edge_test_loader = DataLoader(edge_test_sets, batch_size = 32, sampler = random_test_list)

In [ ]:
# 예시 모델 : 인터넷에서 찾아서 적당히 코드 수정해야할듯!
model = torch.nn.Sequential(
    tl.Conv(64),  # specify ONLY out_channels
    tl.ReLU(),  # use torch.nn wherever you wish
    tl.BatchNorm(),  # BatchNormNd inferred from input
    tl.Conv(128),  # Default kernel_size equal to 3
    tl.ReLU(),
    tl.MaxPool(),
    tl.Conv(256, kernel_size=11),  # "same" padding as default
    # tl.ReLU(),
    tl.GlobalMaxPool(),
    tl.Linear(51),  # Output for 51 classes
)
model = tl.build(model, torch.randn(1, 1, 120, 120)) 
# gray scale 해서 1, 1, 120, 120
model.cuda()

In [ ]:
def accuracy(output, target, topk=(1,)):
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)
 
        _, pred = output.topk(maxk, 1, True, True) #(52, 32)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred)) # (5,32)
 
        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [ ]:
#모델 저장
torch.save(_, '모델 명(1)정확도 1(5)정확도 5.pt')
# model1 = torch.load('model1.pt')

In [ ]:
#one_way_edge_model 모델 학습
import torch.optim as optim
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, betas=(0.9, 0.99), eps=1e-08, weight_decay=0, amsgrad=False)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[2,4,6,8], gamma=0.1)

model.train()
print(len(edge_train_loader))
for epoch in range(10):
    for index, (data, target) in enumerate(edge_train_loader):
        data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if index % 10 == 0:
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            print("acc1 : {}, acc5 : {}".format(acc1[0], acc5[0]))
            print("loss of {} epoch, {} index : {}".format(epoch, index, loss.item()))
            
    scheduler.step()
    print("-----")

In [ ]:
# one_way_edge_model 테스트
model.eval()
correct = 0
total = 0
class_correct = np.zeros(51)
class_total = np.zeros(51)
count = 0
print(len(edge_test_loader))
print(len(labels))
with torch.no_grad():
    for (images, targets) in edge_test_loader:
        images = images.cuda()
        targets = targets.cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        acc1, acc5 = accuracy(outputs, targets, topk=(1, 5))
        print("acc1 : {}, acc5 : {}".format(acc1[0], acc5[0]))
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
        c = (predicted == targets).squeeze()
        count += 1
        for i in range(32):
            target = targets[i]
            class_correct[target] += c[i].item()
            class_total[target] += 1
        if count > len(edge_test_loader)/10:
            break
            
for i in range(51):
    print('Accuract of %5s : %2d %%' %(
        labels[i], 100 * class_correct[i] / class_total[i]))
        
print('Accuracy of the network on the test images: %d %%' % (
      100 * correct / total))